### The Buisness Problem
Joe is in hte U.S. Navy and has been stationed with his family in Newport County, Rhode Island. Having no idea about the area, Joe found a place in the Melville area of Portsmouth, RI on the same Island as the base.  After visiting the area before moving in, Joe found that he prefers the Newport area due to its old historic neighborhood and scenic beaches, but his rental that he picked out is very close to the elementary school and he would like to live near an elementary school.  

Joe wants to find a simiular neighborhood, close to an elementary school and much closer to the First and Second town beaches on southern aquidneck island.



In [1]:
import pandas as pd
import requests
import numpy as np
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
import json
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans


### I descided to take the list of Elementary schools in Rhode Island from Wikipedia and only add the ones from Newport County to my data frame.   Using Geopy, and reading about hte schools on thier wepages, I found some of hte info on the wiki to be incorrect, such as school no longer being open.  To deal with this I changed the name of the school to the city name and dropped some reduntant rows where the same name appeared.  I was left with 15 neighborhoods. 

In [2]:
wiki_url= "https://en.wikipedia.org/wiki/List_of_Rhode_Island_schools"
scdf=pd.read_html(wiki_url, match="Elementary")
scdf=scdf[0]
scdf=scdf[scdf.County.str.contains("Newport")]
scdf.reset_index(drop =True, inplace=True)
scdf.drop(["County", "District", "Students"], axis=1, inplace=True)
newp_df = scdf['City'].apply(lambda x: pd.Series(x.split(', ')))
newp_df=scdf.join(newp_df)
newp_df.drop("City", axis = 1, inplace=True)
newp_df.rename(columns={0: "City", 1: "State", "School Name": "School"}, inplace = True)
newp_df['School'].replace(to_replace=[
    'Jamestown School-Melrose','Wilbur & McMahon Schools', 'Ranger School', 'Sullivan School', 'Linden School'],
    value=['Jamestown','Little Compton', 'Tiverton', 'Newport', 'Middletown'],inplace=True)
newp_df.drop([2, 8, 16], axis=0,inplace=True)
newp_df.reset_index(inplace=True, drop=True)
newp_df

,School,City,State
0,Aquidneck School,Middletown,Rhode Island
1,Carey School,Newport,Rhode Island
2,Elmhurst School,Portsmouth,Rhode Island
3,Forest Avenue School,Middletown,Rhode Island
4,Fort Barton School,Tiverton,Rhode Island
5,Howard Hathaway School,Portsmouth,Rhode Island
6,Jamestown,Jamestown,Rhode Island
7,Middletown,Middletown,Rhode Island
8,Melville School,Portsmouth,Rhode Island
9,Nonquit School,Tiverton,Rhode Island


### I used geopy to find the latitude and longitude for each neighborhood and plotted them on a map.

In [3]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="greg_app")
p= [(41.0,-71.5), (42,-70.0)]

tmp = newp_df.copy()

latlon = tmp.School.apply(lambda addr: geolocator.geocode(addr, viewbox=p, bounded=True))

tmp["Latitude"] = [x.latitude for x in latlon]
tmp["Longitude"] = [x.longitude for x in latlon]
tmp

,School,City,State,Latitude,Longitude
0,Aquidneck School,Middletown,Rhode Island,41.496769,-71.280883
1,Carey School,Newport,Rhode Island,41.474825,-71.311717
2,Elmhurst School,Portsmouth,Rhode Island,41.544269,-71.243381
3,Forest Avenue School,Middletown,Rhode Island,41.525649,-71.287157
4,Fort Barton School,Tiverton,Rhode Island,41.624943,-71.209562
5,Howard Hathaway School,Portsmouth,Rhode Island,41.616213,-71.243937
6,Jamestown,Jamestown,Rhode Island,41.497047,-71.367276
7,Middletown,Middletown,Rhode Island,41.545658,-71.291439
8,Melville School,Portsmouth,Rhode Island,41.587046,-71.283383
9,Nonquit School,Tiverton,Rhode Island,41.571770,-71.192546


In [4]:
# create map  using latitude and longitude values
map_newp = folium.Map(location=[41.496769,-71.280883], zoom_start=10)

# add markers to map
for lat, lng, city, school in zip(tmp['Latitude'], tmp['Longitude'], tmp['City'], tmp['School']):
    label = '{}, {}'.format(school, city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newp)  
    
map_newp

### As you can see Melville is closer to the north of the island, with the beaches Joe wishes to live near on ths southern shores of the island.  There seems to be 3-4 more neighborhoods closer to the beach, so I just needed to see if they are similar in the type of venues.  I will use K means clustering to find similar neighborhoods.

In [5]:
CLIENT_ID = 'KMED0QMTLPT5QNS3D4DKJ15L2JQLVNB3OHEXMETVS4LP0XPC' # your Foursquare ID
CLIENT_SECRET = 'B5OPZZ2DH3FCFDGKFFMOWKEBREYMF1Z3CRXKWZFXJQMNJBIY' # your Foursquare Secret
VERSION = '20201015' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KMED0QMTLPT5QNS3D4DKJ15L2JQLVNB3OHEXMETVS4LP0XPC
CLIENT_SECRET:B5OPZZ2DH3FCFDGKFFMOWKEBREYMF1Z3CRXKWZFXJQMNJBIY


In [6]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Child care: 5744ccdfe4b0c0459246b4c7
DayCare: 4f4532974b9074f6e4fb0104
Nursery School: 4f4533814b9074f6e4fb0107
Preschool: 52e81612bcbc57f1066b7a45

In [7]:
def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius,
            LIMIT,
            )
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['School', 
                  'School Latitude', 
                  'School Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',        
                  'Venue Category']
    
    return(nearby_venues)

In [8]:
Newport_venues = getNearbyVenues(names=tmp['School'],
                                   latitudes=tmp['Latitude'],
                                   longitudes=tmp['Longitude']
                                  )

Aquidneck School
Carey School
Elmhurst School
Forest Avenue School
Fort Barton School
Howard Hathaway School
Jamestown
Middletown
Melville School
Nonquit School
Pocasset School
Prudence Island School
Tiverton
Newport
Little Compton


In [9]:
print(Newport_venues.shape)
Newport_venues.head()

(521, 7)


,School,School Latitude,School Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Aquidneck School,41.496769,-71.280883,Newport Beach,41.488864,-71.281716,Beach
1,Aquidneck School,41.496769,-71.280883,Atlantic Grille,41.492130,-71.284040,Diner
2,Aquidneck School,41.496769,-71.280883,Newport County YMCA,41.498360,-71.286326,Gym
3,Aquidneck School,41.496769,-71.280883,Flo's Clam Shack,41.490032,-71.284916,Seafood Restaurant
4,Aquidneck School,41.496769,-71.280883,Easton's Beach,41.488658,-71.290326,Beach


In [10]:
Newport_venues.groupby('School').count()
print('There are {} uniques categories.'.format(len(Newport_venues['Venue Category'].unique())))

There are 139 uniques categories.


In [11]:
# one hot encoding
Newport_onehot = pd.get_dummies(Newport_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Newport_onehot['School'] = Newport_venues['School'] 



Newport_onehot.head()

,Airport,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Athletics & Sports,Australian Restaurant,Auto Dealership,BBQ Joint,Bagel Shop,...,Video Game Store,Video Store,Vietnamese Restaurant,Vineyard,Warehouse Store,Wine Shop,Winery,Women's Store,Yoga Studio,School
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Aquidneck School
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Aquidneck School
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Aquidneck School
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Aquidneck School
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Aquidneck School


In [12]:
Newport_grouped=Newport_onehot.groupby('School').mean().reset_index()
Newport_grouped

,School,Airport,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Athletics & Sports,Australian Restaurant,Auto Dealership,BBQ Joint,...,Trail,Video Game Store,Video Store,Vietnamese Restaurant,Vineyard,Warehouse Store,Wine Shop,Winery,Women's Store,Yoga Studio
0,Aquidneck School,0.000000,0.063830,0.000000,0.00,0.000000,0.021277,0.00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
1,Carey School,0.000000,0.070000,0.000000,0.01,0.000000,0.000000,0.01,0.000000,0.010000,...,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.01
2,Elmhurst School,0.000000,0.000000,0.000000,0.00,0.000000,0.111111,0.00,0.000000,0.000000,...,0.222222,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.00
3,Forest Avenue School,0.034091,0.000000,0.000000,0.00,0.011364,0.000000,0.00,0.011364,0.011364,...,0.000000,0.011364,0.000000,0.011364,0.011364,0.011364,0.011364,0.011364,0.000000,0.00
4,Fort Barton School,0.000000,0.166667,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,...,0.041667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
5,Howard Hathaway School,0.000000,0.150000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
6,Jamestown,0.000000,0.032258,0.000000,0.00,0.000000,0.032258,0.00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
7,Little Compton,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
8,Melville School,0.000000,0.111111,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.055556,0.00
9,Middletown,0.103448,0.000000,0.000000,0.00,0.034483,0.000000,0.00,0.034483,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00


In [13]:
num_top_venues = 5

for hood in Newport_grouped['School']:
    print("----"+hood+"----")
    temp = Newport_grouped[Newport_grouped['School'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Aquidneck School----
                 venue  freq
0   Seafood Restaurant  0.06
1  American Restaurant  0.06
2          Pizza Place  0.06
3                Beach  0.06
4          Coffee Shop  0.04


----Carey School----
                 venue  freq
0   Seafood Restaurant  0.07
1  American Restaurant  0.07
2                Hotel  0.06
3        Historic Site  0.05
4      Bed & Breakfast  0.04


----Elmhurst School----
                venue  freq
0                Park  0.22
1               Trail  0.22
2  Italian Restaurant  0.11
3  Athletics & Sports  0.11
4         Golf Course  0.11


----Forest Avenue School----
                  venue  freq
0                 Hotel  0.08
1                Bakery  0.03
2        Sandwich Place  0.03
3  Gym / Fitness Center  0.03
4            Donut Shop  0.03


----Fort Barton School----
                 venue  freq
0  American Restaurant  0.17
1      Harbor / Marina  0.12
2                 Café  0.08
3         Liquor Store  0.08
4   Seafood Restaurant  0

In [14]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [15]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['School']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['School'] = Newport_grouped['School']

for ind in np.arange(Newport_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Newport_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,School,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Aquidneck School,Pizza Place,American Restaurant,Seafood Restaurant,Beach,Convenience Store,Gym,Scenic Lookout,Restaurant,Coffee Shop,Snack Place
1,Carey School,American Restaurant,Seafood Restaurant,Hotel,Historic Site,Pub,Bed & Breakfast,Burger Joint,Italian Restaurant,Clothing Store,Café
2,Elmhurst School,Park,Trail,Beach,Vineyard,Italian Restaurant,Athletics & Sports,Golf Course,Yoga Studio,Falafel Restaurant,Flower Shop
3,Forest Avenue School,Hotel,Sandwich Place,Bakery,Coffee Shop,Department Store,Donut Shop,Gym / Fitness Center,Airport,Brewery,Business Service
4,Fort Barton School,American Restaurant,Harbor / Marina,Café,Coffee Shop,Seafood Restaurant,Liquor Store,Bar,French Restaurant,Breakfast Spot,Massage Studio
5,Howard Hathaway School,American Restaurant,Convenience Store,Pizza Place,Breakfast Spot,Café,Fast Food Restaurant,Liquor Store,Beach,Bar,Donut Shop
6,Jamestown,Harbor / Marina,Ice Cream Shop,Hotel,Convenience Store,Golf Course,City,Seafood Restaurant,Hardware Store,Liquor Store,Breakfast Spot
7,Little Compton,Pizza Place,Gift Shop,Breakfast Spot,Post Office,Thrift / Vintage Store,Café,Fish Market,Fast Food Restaurant,Farm,Falafel Restaurant
8,Melville School,Harbor / Marina,American Restaurant,Pizza Place,Museum,Café,Campground,Garden,Park,Lingerie Store,Light Rail Station
9,Middletown,Airport,Sporting Goods Shop,Supplement Shop,Shoe Store,Pet Store,Other Great Outdoors,Italian Restaurant,Grocery Store,Golf Course,Gas Station


In [16]:
# set number of clusters
kclusters = 5

Newport_grouped_clustering = Newport_grouped.drop('School', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Newport_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 2, 0, 0, 0, 0, 3, 0, 0])

In [19]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Newport_merged = tmp

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Newport_merged = Newport_merged.join(neighborhoods_venues_sorted.set_index('School'), on='School')

Newport_merged

,School,City,State,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Aquidneck School,Middletown,Rhode Island,41.496769,-71.280883,0,Pizza Place,American Restaurant,Seafood Restaurant,Beach,Convenience Store,Gym,Scenic Lookout,Restaurant,Coffee Shop,Snack Place
1,Carey School,Newport,Rhode Island,41.474825,-71.311717,0,American Restaurant,Seafood Restaurant,Hotel,Historic Site,Pub,Bed & Breakfast,Burger Joint,Italian Restaurant,Clothing Store,Café
2,Elmhurst School,Portsmouth,Rhode Island,41.544269,-71.243381,2,Park,Trail,Beach,Vineyard,Italian Restaurant,Athletics & Sports,Golf Course,Yoga Studio,Falafel Restaurant,Flower Shop
3,Forest Avenue School,Middletown,Rhode Island,41.525649,-71.287157,0,Hotel,Sandwich Place,Bakery,Coffee Shop,Department Store,Donut Shop,Gym / Fitness Center,Airport,Brewery,Business Service
4,Fort Barton School,Tiverton,Rhode Island,41.624943,-71.209562,0,American Restaurant,Harbor / Marina,Café,Coffee Shop,Seafood Restaurant,Liquor Store,Bar,French Restaurant,Breakfast Spot,Massage Studio
5,Howard Hathaway School,Portsmouth,Rhode Island,41.616213,-71.243937,0,American Restaurant,Convenience Store,Pizza Place,Breakfast Spot,Café,Fast Food Restaurant,Liquor Store,Beach,Bar,Donut Shop
6,Jamestown,Jamestown,Rhode Island,41.497047,-71.367276,0,Harbor / Marina,Ice Cream Shop,Hotel,Convenience Store,Golf Course,City,Seafood Restaurant,Hardware Store,Liquor Store,Breakfast Spot
7,Middletown,Middletown,Rhode Island,41.545658,-71.291439,0,Airport,Sporting Goods Shop,Supplement Shop,Shoe Store,Pet Store,Other Great Outdoors,Italian Restaurant,Grocery Store,Golf Course,Gas Station
8,Melville School,Portsmouth,Rhode Island,41.587046,-71.283383,0,Harbor / Marina,American Restaurant,Pizza Place,Museum,Café,Campground,Garden,Park,Lingerie Store,Light Rail Station
9,Nonquit School,Tiverton,Rhode Island,41.571770,-71.192546,4,Deli / Bodega,Cheese Shop,American Restaurant,Art Gallery,Arts & Crafts Store,Ice Cream Shop,Flower Shop,Gift Shop,Farm,Food & Drink Shop


In [20]:
# create map
map_clusters = folium.Map(location=[41.496769,-71.280883], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Newport_merged['Latitude'], Newport_merged['Longitude'], Newport_merged['School'], Newport_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Cluster 0 List


In [21]:
Newport_merged.loc[Newport_merged['Cluster Labels'] == 0, Newport_merged.columns[[0] + list(range(5, Newport_merged.shape[1]))]]

,School,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Aquidneck School,0,Pizza Place,American Restaurant,Seafood Restaurant,Beach,Convenience Store,Gym,Scenic Lookout,Restaurant,Coffee Shop,Snack Place
1,Carey School,0,American Restaurant,Seafood Restaurant,Hotel,Historic Site,Pub,Bed & Breakfast,Burger Joint,Italian Restaurant,Clothing Store,Café
3,Forest Avenue School,0,Hotel,Sandwich Place,Bakery,Coffee Shop,Department Store,Donut Shop,Gym / Fitness Center,Airport,Brewery,Business Service
4,Fort Barton School,0,American Restaurant,Harbor / Marina,Café,Coffee Shop,Seafood Restaurant,Liquor Store,Bar,French Restaurant,Breakfast Spot,Massage Studio
5,Howard Hathaway School,0,American Restaurant,Convenience Store,Pizza Place,Breakfast Spot,Café,Fast Food Restaurant,Liquor Store,Beach,Bar,Donut Shop
6,Jamestown,0,Harbor / Marina,Ice Cream Shop,Hotel,Convenience Store,Golf Course,City,Seafood Restaurant,Hardware Store,Liquor Store,Breakfast Spot
7,Middletown,0,Airport,Sporting Goods Shop,Supplement Shop,Shoe Store,Pet Store,Other Great Outdoors,Italian Restaurant,Grocery Store,Golf Course,Gas Station
8,Melville School,0,Harbor / Marina,American Restaurant,Pizza Place,Museum,Café,Campground,Garden,Park,Lingerie Store,Light Rail Station
10,Pocasset School,0,American Restaurant,Pharmacy,Pizza Place,Convenience Store,Food,Coffee Shop,Portuguese Restaurant,Discount Store,Chinese Restaurant,Donut Shop
12,Tiverton,0,Sandwich Place,Ice Cream Shop,Donut Shop,Playground,Spa,Portuguese Restaurant,Chinese Restaurant,Coffee Shop,American Restaurant,Dessert Shop


### The map and cluster zero shows that The Melville, Newport, Carey School, and Aquidneck Schools fall into the same cluster so they are similar.  In the analysis I will further compare thre neighborhoods, and find the distances to various locations Joe has specified.